In [ ]:
!pip install emoji
import nltk
from nltk.corpus import stopwords # Stopwords
import emoji
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 8.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import pandas as pd

df = pd.read_json('combined_dataset.jsonl', lines=True)
data= df[['paragraph','label']]
data

,paragraph,label
0,ندوة «من سلب اسكندرون إلى سلب فلسطين» على مدرج...,true
1,بعد استقلال الجزائر، تزوجت جميلة بوحيرد بالمحا...,true
2,وفي الوقت الذي لم تفصح فيه الخاطر عن فحوى ما ح...,true
3,"وبدورها، كتبت نور الهجري، على فيسبوك "" احتفال ...",true
4,وظل بوتفليقة في سدة الرئاسة الجزائرية نحو 20 ع...,true
...,...,...
7995,أجرى رئيس الأركان العامة للقوات المسلحة الروسي...,false
7996,ومنعت جبهة تحرير شعب تيغراي يوم الجمعة الماضي ...,false
7997,تتميز هونغ كونغ بالعديد من المعالم السياحية وا...,false
7998,وتابع «بعدما تفرقت آراء الصحابة ولم يتفقوا على...,false


In [ ]:
print(data.info())
print(data.label.value_counts())
print(data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   paragraph  8000 non-null   object
 1   label      8000 non-null   object
dtypes: object(2)
memory usage: 125.1+ KB
None
label
true     5034
false    2966
Name: count, dtype: int64
paragraph    0
label        0
dtype: int64


In [ ]:
arabic_stopwords = stopwords.words("arabic")
len(arabic_stopwords)

754

In [ ]:
data['stop_words'] = data['paragraph'].apply(lambda x: len(set(x.split()) & set(arabic_stopwords)))
data.stop_words.value_counts()

<ipython-input-5-7a7f31435d1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stop_words'] = data['paragraph'].apply(lambda x: len(set(x.split()) & set(arabic_stopwords)))


stop_words
4     1119
5     1086
3     1079
6      940
7      792
2      670
8      601
9      394
10     333
1      315
11     207
12     148
0       98
13      91
14      54
15      31
16      20
17      11
18       6
20       3
19       1
24       1
Name: count, dtype: int64

In [ ]:
!wget https://raw.githubusercontent.com/alaa-a-a/multi-dialect-arabic-stop-words/main/Stop-words/stop_list_1177.txt
arabic_stop_words = []
with open ('./stop_list_1177.txt',encoding='utf-8') as f :
    for word in f.readlines() :
        arabic_stop_words.append(word.split("\n")[0])

--2024-07-26 16:26:06--  https://raw.githubusercontent.com/alaa-a-a/multi-dialect-arabic-stop-words/main/Stop-words/stop_list_1177.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11468 (11K) [text/plain]
Saving to: ‘stop_list_1177.txt’

stop_list_1177.txt  100%[===================>]  11.20K  --.-KB/s    in 0.001s  

2024-07-26 16:26:06 (19.1 MB/s) - ‘stop_list_1177.txt’ saved [11468/11468]



In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.isri import ISRIStemmer
import string
import re
from bs4 import BeautifulSoup
nltk.download('stopwords')


tok = WordPunctTokenizer()

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    return re.sub(arabic_diacritics, '', text)


def remove_punctuations(text):
    arabic_punctuations = '''`÷×؛<><<>>_«»()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
    english_punctuations = string.punctuation
    punctuations_list = arabic_punctuations + english_punctuations
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    # return re.sub(r'(.)\1+', r'\1', text)     # keep only 1 repeat
    return re.sub(r'(.)\1+', r'\1\1', text)  # keep 2 repeat

def remove_stop_words(text):
    word_list = nltk.tokenize.wordpunct_tokenize(text.lower())
    word_list = [ w for w in word_list if not w in arabic_stop_words]
    return (" ".join(word_list)).strip()



def remove_non_arabic_letters(text):
    text = re.sub(r'([@A-Za-z0-9_]+)|#|http\S+', ' ', text) # removes non arabic letters
    text = re.sub(r'ـــــــــــــ', '', text) # removes non arabic letters
    return text


def clean_str(text):
    text = remove_non_arabic_letters(text)
    text = remove_punctuations(text)
    text = remove_diacritics(text)
    text = remove_repeating_char(text)
    text = remove_stop_words(text)

    # Extract text from HTML tags
    soup = BeautifulSoup(text, 'lxml')
    text = soup.get_text()

    words = tok.tokenize(text)
    return (" ".join(words)).strip()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data["text_clean"] = data["paragraph"].apply(lambda x: clean_str(x))

data.head()

<ipython-input-8-e8ae124534f5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text_clean"] = data["paragraph"].apply(lambda x: clean_str(x))


,paragraph,label,stop_words,text_clean
0,ندوة «من سلب اسكندرون إلى سلب فلسطين» على مدرج...,true,3,ندوة سلب اسكندرون سلب فلسطين مدرج دار البعث دم...
1,بعد استقلال الجزائر، تزوجت جميلة بوحيرد بالمحا...,true,7,استقلال الجزائر تزوجت جميلة بوحيرد بالمحامي ال...
2,وفي الوقت الذي لم تفصح فيه الخاطر عن فحوى ما ح...,true,8,تفصح الخاطر فحوى حمله وزير الخارجية الكويتي أح...
3,"وبدورها، كتبت نور الهجري، على فيسبوك "" احتفال ...",true,6,وبدورها كتبت نور الهجري فيسبوك احتفال بعيد الأ...
4,وظل بوتفليقة في سدة الرئاسة الجزائرية نحو 20 ع...,true,4,وظل بوتفليقة سدة الرئاسة الجزائرية يظهر علني ن...


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("routellm/bert_gpt4_augmented")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
data['tokens'] = data['text_clean'].apply(lambda x: tokenizer.tokenize(x))

<ipython-input-10-a317e5c288b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tokens'] = data['text_clean'].apply(lambda x: tokenizer.tokenize(x))


In [ ]:
data['tokens']


0       [▁, ندو, ة, ▁سلب, ▁اس, ك, ندر, ون, ▁سلب, ▁فلسط...
1       [▁استقلال, ▁الجزائر, ▁, تزوج, ت, ▁جميلة, ▁بو, ...
2       [▁تف, صح, ▁الخ, ا, طر, ▁, فح, وى, ▁حمله, ▁وزير...
3       [▁وب, دور, ها, ▁كتبت, ▁نور, ▁اله, جري, ▁, فيسب...
4       [▁و, ظل, ▁, بوت, فل, يقة, ▁سد, ة, ▁الرئاسة, ▁ا...
                              ...                        
7995    [▁أ, جرى, ▁رئيس, ▁الأر, كان, ▁العامة, ▁للقوات,...
7996    [▁ومن, عت, ▁, جبهة, ▁تحرير, ▁شعب, ▁تي, غر, اي,...
7997    [▁ت, تميز, ▁هو, نغ, ▁كون, غ, ▁ب, العديد, ▁الم,...
7998    [▁وتابع, ▁بعدما, ▁تفر, قت, ▁آر, اء, ▁ال, صحاب,...
7999    [▁الوزارة, ▁بيان, ▁الإصابة, ▁الم, ؤكد, ة, ▁مست...
Name: tokens, Length: 8000, dtype: object

In [ ]:
label_mapping = {'true': 1, 'false': 0}
data['label'] = data['label'].map(label_mapping)

<ipython-input-12-be5bee8ab2e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = data['label'].map(label_mapping)


In [ ]:
data['label']

0       1
1       1
2       1
3       1
4       1
       ..
7995    0
7996    0
7997    0
7998    0
7999    0
Name: label, Length: 8000, dtype: int64

In [ ]:
# Split the data
train_size = 6002
val_size = 672
test_size = 1326

train_data = data[:train_size]
val_data = data[train_size:train_size+val_size]
test_data = data[train_size+val_size:train_size+val_size+test_size]

# Now train_data, val_data, and test_data are variables containing the respective splits
print(f'Train data size: {len(train_data)}')
print(f'Validation data size: {len(val_data)}')
print(f'Test data size: {len(test_data)}')

Train data size: 6002
Validation data size: 672
Test data size: 1326


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("routellm/bert_gpt4_augmented", num_labels=2,ignore_mismatched_sizes=True)


config.json:   0%|          | 0.00/924 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at routellm/bert_gpt4_augmented and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def format_dataset(encodings, labels):
    return {
        'input_ids': torch.tensor(encodings['input_ids']),
        'attention_mask': torch.tensor(encodings['attention_mask']),
        'labels': torch.tensor(labels.values, dtype=torch.long)  # <-- Converting labels to torch long type
    }

train_encodings = tokenizer(train_data['text_clean'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_data['text_clean'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_data['text_clean'].tolist(), truncation=True, padding=True)


train_dataset = format_dataset(train_encodings, train_data['label'])
val_dataset = format_dataset(val_encodings, val_data['label'])
test_dataset = format_dataset(test_encodings, test_data['label'])

In [ ]:
def format_dataset(encodings, labels):
    return {
        'input_ids': torch.tensor(encodings['input_ids']),
        'attention_mask': torch.tensor(encodings['attention_mask']),
        'labels': torch.tensor(labels.values, dtype=torch.long)  # <-- Converting labels to torch long type
    }

# Convert the formatted datasets to PyTorch Datasets
import torch

class PropagandaDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train_data['text_clean'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_data['text_clean'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_data['text_clean'].tolist(), truncation=True, padding=True)

# Use the custom Dataset class
train_dataset = PropagandaDataset(train_encodings, train_data['label'].values)
val_dataset = PropagandaDataset(val_encodings, val_data['label'].values)
test_dataset = PropagandaDataset(test_encodings, test_data['label'].values)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir='./output',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to="tensorboard",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Define Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda pred: {"accuracy": accuracy_score(val_dataset.labels, pred.predictions.argmax(axis=1)),
                                  "precision_recall_fscore": precision_recall_fscore_support(val_dataset.labels, pred.predictions.argmax(axis=1), average='binary')[:3]}
)
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()

print(f"Accuracy: {eval_results['eval_accuracy']}")
print(f"Precision, Recall, F1-score: {eval_results['eval_precision_recall_fscore']}")

# Save the fine-tuned model
model_path = "./fine_tuned_arabert_propaganda_detection"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

print(f"Model saved to {model_path}")

Epoch,Training Loss,Validation Loss,Accuracy,Precision Recall Fscore
1,0.647500,0.654052,0.632440,"(0.6324404761904762, 1.0, 0.7748404740200546)"
2,0.640200,0.600508,0.696429,"(0.6983842010771992, 0.9152941176470588, 0.7922606924643584)"
3,0.611600,0.591590,0.691964,"(0.7088122605363985, 0.8705882352941177, 0.781414994720169)"


Trainer is attempting to log a value of "(0.6324404761904762, 1.0, 0.7748404740200546)" of type <class 'tuple'> for key "eval/precision_recall_fscore" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(0.6983842010771992, 0.9152941176470588, 0.7922606924643584)" of type <class 'tuple'> for key "eval/precision_recall_fscore" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "(0.7088122605363985, 0.8705882352941177, 0.781414994720169)" of type <class 'tuple'> for key "eval/precision_recall_fscore" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Trainer is attempting to log a value of "(0.7088122605363985, 0.8705882352941177, 0.781414994720169)" of type <class 'tuple'> for key "eval/precision_recall_fscore" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Accuracy: 0.6919642857142857
Precision, Recall, F1-score: (0.7088122605363985, 0.8705882352941177, 0.781414994720169)
Model saved to ./fine_tuned_arabert_propaganda_detection


In [ ]:
# prompt: i want to make test

from torch.utils.data import DataLoader

# Assuming you have your test_dataset ready
test_dataloader = DataLoader(test_dataset, batch_size=16)

# Put model in evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

predictions = []
true_labels = []

with torch.no_grad():
  for batch in test_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    # Get predicted labels
    predicted_labels = torch.argmax(logits, dim=1)
    predictions.extend(predicted_labels.tolist())
    true_labels.extend(labels.tolist())

# Calculate accuracy and other metrics
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')

print(f"Test Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
